## CDS Assignment

#### Dataframe Creation

In [74]:
import pandas as pd
import numpy as np
import re

In [75]:
# instantiate main dataframe, df

df = pd.DataFrame()

df['Dx ID'] = 0
df['Mx ID'] = 0
df['Dx Mx Association'] = 0
df['Dx Name'] = 0
df['Dx Prevalence'] = 0
df['Mx Name'] = 0
df['Mx PPV'] = 0
df['Mx Sensitivity'] = 0
df['Mx Import'] = 0
df['Specificity'] = 0
df['LR (+)'] = 0
df['LR (-)'] = 0

df

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Dx Prevalence,Mx Name,Mx PPV,Mx Sensitivity,Mx Import,Specificity,LR (+),LR (-)


In [76]:
# incorporate the associations between Dx and Mx

df_DxMx = pd.read_csv('dx_mx_associations.txt', sep='\t', names=['dx', 'mx'])

df_DxMx.head()

,dx,mx
0,DX 362 PEPTIC ULCER,MX 34 59 ABDOMEN PAIN EPIGASTRIUM RECURRENT AT...
1,DX 362 PEPTIC ULCER,MX 22 62 ABDOMEN PAIN EPIGASTRIUM SEASONAL HX
2,DX 362 PEPTIC ULCER,MX 23 64 ABDOMEN PAIN EXACERBATION WITH ALCOHOL
3,DX 362 PEPTIC ULCER,MX 02 198 AGE 16 TO 25
4,DX 362 PEPTIC ULCER,MX 03 199 AGE 26 TO 55


In [77]:
# read Dx Mx file

f = open('dx_mx_associations.txt')
lines = f.readlines()
f.close

lines[:5]

['DX 362  PEPTIC ULCER \tMX 34 59 ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX\n',
 'DX 362  PEPTIC ULCER \tMX 22 62 ABDOMEN PAIN EPIGASTRIUM SEASONAL HX\n',
 'DX 362  PEPTIC ULCER \tMX 23 64 ABDOMEN PAIN EXACERBATION WITH ALCOHOL\n',
 'DX 362  PEPTIC ULCER \tMX 02 198 AGE 16 TO 25\n',
 'DX 362  PEPTIC ULCER \tMX 03 199 AGE 26 TO 55\n']

In [78]:
# group the Dx ID, Dx name, Dx prevalence, Mx ID, Mx name, Mx PPV, and Mx Sensitivity

m = re.compile(r'DX\s(\d+)\s+(.+)\s\sMX\s(\d)(\d)\s(\d+)\s(.+)')

dx_id = []
dx_name = []
mx_id = []
mx_name = []
mx_ppv = []
mx_sens = []

for i in range(len(lines)):
    dx_id.append((m.match(lines[i]).group(1)))
    dx_name.append((m.match(lines[i]).group(2)))
    mx_id.append((m.match(lines[i]).group(5)))
    mx_name.append((m.match(lines[i]).group(6)))
    mx_ppv.append((m.match(lines[i]).group(3)))
    mx_sens.append((m.match(lines[i]).group(4)))

In [79]:
# combine Dx and Mx IDs for associations column

dx_mx = []

for i in range(len(dx_id)):
    dx_mx.append(dx_id[i] + ', ' + mx_id[i])
    
dx_mx[:5]

['362, 59', '362, 62', '362, 64', '362, 198', '362, 199']

In [80]:
# incorporate prevalence table

df_prev = pd.read_csv('dx_prev.txt', sep='\t')

df_prev

,id,prev,name
0,114,3,choledocholithiasis
1,362,5,peptic ulcer
2,441,2,pyloric obstruction
3,187,3,gastric carcinoma
4,189,2,gastritis giant hypertrophic alias menetriers_...
5,351,4,pancreatitis acute
6,352,3,pancreatitis chronic
7,188,2,gastric lymphoma
8,192,2,gastrointestinal sarcoidosis
9,616,1,anisakiasis


In [81]:
# incorporate import numbers

f = open('mx.txt')
lines = f.readlines()

lines[:5]

['MX 100 ABDOMEN SURGERY RECENT HX    IM  1 TY 1 \n',
 'MX 1001 COLON BARIUM ENEMA WALL THICKENED   IM  3 TY 5 \n',
 'MX 1005 COLONOSCOPY BIOPSY GRANULOMA S NON CASEATING   IM  4 TY 6 \n',
 'MX 1012 COLONOSCOPY MULTIPLE NODULAR FILLING DEFECT S   IM  4 TY 6 \n',
 'MX 102 ABDOMEN TENDERNESS EPIGASTRIUM NON HEPATIC   IM  3 TY 3 \n']

In [82]:
# group the Mx ID numbers and import numbers

m = re.compile(r'MX\s(\d+).+IM\s+(\d)')

mx_import_id = []
mx_import_num = []

for i in range(len(lines)):
    mx_import_id.append((m.match(lines[i]).group(1)))
    mx_import_num.append((m.match(lines[i]).group(2)))

In [83]:
# create dataframe for imports

df_mx_import = pd.DataFrame()

df_mx_import['Mx ID'] = mx_import_id
df_mx_import['Import Num'] = mx_import_num

In [84]:
df_mx_import.tail()

,Mx ID,Import Num
348,980,3
349,982,4
350,985,3
351,989,4
352,998,3


#### Add everything to df

In [85]:
# put associations, Dx name, Mx name, PPV, and sensitivity into df

df['Dx ID'] = dx_id
df['Mx ID'] = mx_id
df['Dx Mx Association'] = dx_mx
df['Dx Name'] = dx_name
df['Mx Name'] = mx_name
df['Mx PPV'] = mx_ppv
df['Mx Sensitivity'] = mx_sens

In [86]:
# add import numbers

for i in range(len(df)):
    for j in range(len(df_mx_import)):
        if df['Mx ID'][i] == mx_import_id[j]:
            df['Mx Import'][i] = mx_import_num[j]

C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [87]:
# add prevalences

for i in range(len(df)):
    for j in range(len(df_prev)):
        if int(df['Dx ID'][i]) == df_prev['id'][j]:
            df['Dx Prevalence'][i] = df_prev['prev'][j]

C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [88]:
df.head()

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Dx Prevalence,Mx Name,Mx PPV,Mx Sensitivity,Mx Import,Specificity,LR (+),LR (-)
0,362,59,"362, 59",PEPTIC ULCER,5.0,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,3,4,2.0,NaN,NaN,NaN
1,362,62,"362, 62",PEPTIC ULCER,5.0,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,2,2,2.0,NaN,NaN,NaN
2,362,64,"362, 64",PEPTIC ULCER,5.0,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,2,3,2.0,NaN,NaN,NaN
3,362,198,"362, 198",PEPTIC ULCER,5.0,AGE 16 TO 25,0,2,3.0,NaN,NaN,NaN
4,362,199,"362, 199",PEPTIC ULCER,5.0,AGE 26 TO 55,0,3,3.0,NaN,NaN,NaN


In [89]:
# cast all numeric values to integers 

for i in range(len(df)):  
    df['Dx Prevalence'][i] = int(df['Dx Prevalence'][i])    
    df['Mx PPV'][i] = int(df['Mx PPV'][i])    
    df['Mx Sensitivity'][i] = int(df['Mx Sensitivity'][i])    
    df['Mx Import'][i] = int(df['Mx Import'][i])    

df.head()

C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWi

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Dx Prevalence,Mx Name,Mx PPV,Mx Sensitivity,Mx Import,Specificity,LR (+),LR (-)
0,362,59,"362, 59",PEPTIC ULCER,5.0,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,3,4,2.0,NaN,NaN,NaN
1,362,62,"362, 62",PEPTIC ULCER,5.0,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,2,2,2.0,NaN,NaN,NaN
2,362,64,"362, 64",PEPTIC ULCER,5.0,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,2,3,2.0,NaN,NaN,NaN
3,362,198,"362, 198",PEPTIC ULCER,5.0,AGE 16 TO 25,0,2,3.0,NaN,NaN,NaN
4,362,199,"362, 199",PEPTIC ULCER,5.0,AGE 26 TO 55,0,3,3.0,NaN,NaN,NaN


In [91]:
# change prevalence, ppv, and sensitivity to proportions

for i in range(len(df)):
    # prevalences (1 - 5)
    if df['Dx Prevalence'][i] == 1:
        df['Dx Prevalence'][i] = 1/100000
    if df['Dx Prevalence'][i] == 2:
        df['Dx Prevalence'][i] = 1/10000
    if df['Dx Prevalence'][i] == 3:
        df['Dx Prevalence'][i] = 1/100
    if df['Dx Prevalence'][i] == 4:
        df['Dx Prevalence'][i] = 1/10
    if df['Dx Prevalence'][i] == 5:
        df['Dx Prevalence'][i] = 1/2
    # PPVs (0 - 5)
    if df['Mx PPV'][i] == 0:
        df['Mx PPV'][i] = 0.0001
    if df['Mx PPV'][i] == 1:
        df['Mx PPV'][i] = 0.2
    if df['Mx PPV'][i] == 2:
        df['Mx PPV'][i] = 0.4
    if df['Mx PPV'][i] == 3:
        df['Mx PPV'][i] = 0.6
    if df['Mx PPV'][i] == 4:
        df['Mx PPV'][i] = 0.8
    if df['Mx PPV'][i] == 5:
        df['Mx PPV'][i] = 1
    # sensitivities (1 - 5)
    if df['Mx Sensitivity'][i] == 1:
        df['Mx Sensitivity'][i] = 0.0001
    if df['Mx Sensitivity'][i] == 2:
        df['Mx Sensitivity'][i] = 0.25
    if df['Mx Sensitivity'][i] == 3:
        df['Mx Sensitivity'][i] = 0.5
    if df['Mx Sensitivity'][i] == 4:
        df['Mx Sensitivity'][i] = 0.75
    if df['Mx Sensitivity'][i] == 5:
        df['Mx Sensitivity'][i] = 1
   
df.head()

C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be se

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Dx Prevalence,Mx Name,Mx PPV,Mx Sensitivity,Mx Import,Specificity,LR (+),LR (-)
0,362,59,"362, 59",PEPTIC ULCER,0.5,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.6,0.75,2.0,NaN,NaN,NaN
1,362,62,"362, 62",PEPTIC ULCER,0.5,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.4,0.25,2.0,NaN,NaN,NaN
2,362,64,"362, 64",PEPTIC ULCER,0.5,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.4,0.5,2.0,NaN,NaN,NaN
3,362,198,"362, 198",PEPTIC ULCER,0.5,AGE 16 TO 25,0.0001,0.25,3.0,NaN,NaN,NaN
4,362,199,"362, 199",PEPTIC ULCER,0.5,AGE 26 TO 55,0.0001,0.5,3.0,NaN,NaN,NaN


In [92]:
# add specificity helper function

def calc_specificity(sens, ppv, prev):
    """
    Calculate specificity based on sensitivity, positive
    predictive value, & prevalence. 
    """
    spec = 1 - ((sens*prev/ppv - sens*prev)/(1-prev))
    return spec

In [97]:
# calculate specificities

for i in range(len(df)):
    df['Specificity'][i] = calc_specificity(df['Mx Sensitivity'][i], df['Mx PPV'][i], df['Dx Prevalence'][i])
    
df.head()

C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Dx ID,Mx ID,Dx Mx Association,Dx Name,Dx Prevalence,Mx Name,Mx PPV,Mx Sensitivity,Mx Import,Specificity,LR (+),LR (-)
0,362,59,"362, 59",PEPTIC ULCER,0.5,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.6,0.75,2.0,0.500,NaN,NaN
1,362,62,"362, 62",PEPTIC ULCER,0.5,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.4,0.25,2.0,0.625,NaN,NaN
2,362,64,"362, 64",PEPTIC ULCER,0.5,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.4,0.5,2.0,0.250,NaN,NaN
3,362,198,"362, 198",PEPTIC ULCER,0.5,AGE 16 TO 25,0.0001,0.25,3.0,-2498.750,NaN,NaN
4,362,199,"362, 199",PEPTIC ULCER,0.5,AGE 26 TO 55,0.0001,0.5,3.0,-4998.500,NaN,NaN


In [98]:
for i in range(len(df)):
    df['LR (+)'][i] = df['Mx Sensitivity'][i]/(1-df['Specificity'][i])
    df['LR (-)'][i] = (1-df['Mx Sensitivity'][i])/df['Specificity'][i]

C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Grayson Ruhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered i

In [100]:
df

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Dx Prevalence,Mx Name,Mx PPV,Mx Sensitivity,Mx Import,Specificity,LR (+),LR (-)
0,362,59,"362, 59",PEPTIC ULCER,0.50000,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.6,0.75,2.0,0.500000,1.500000,0.500000
1,362,62,"362, 62",PEPTIC ULCER,0.50000,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.4,0.25,2.0,0.625000,0.666667,1.200000
2,362,64,"362, 64",PEPTIC ULCER,0.50000,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.4,0.5,2.0,0.250000,0.666667,2.000000
3,362,198,"362, 198",PEPTIC ULCER,0.50000,AGE 16 TO 25,0.0001,0.25,3.0,-2498.750000,0.000100,-0.000300
4,362,199,"362, 199",PEPTIC ULCER,0.50000,AGE 26 TO 55,0.0001,0.5,3.0,-4998.500000,0.000100,-0.000100
5,362,200,"362, 200",PEPTIC ULCER,0.50000,AGE GTR THAN 55,0.0001,0.25,3.0,-2498.750000,0.000100,-0.000300
6,362,208,"362, 208",PEPTIC ULCER,0.50000,ALCOHOLISM CHRONIC HX,0.4,0.25,3.0,0.625000,0.666667,1.200000
7,362,391,"362, 391",PEPTIC ULCER,0.50000,ASPIRIN ADMINISTRATION IN LARGE AMOUNT RECENT HX,0.2,0.25,1.0,0.000000,0.250000,inf
8,362,957,"362, 957",PEPTIC ULCER,0.50000,CIGARETTE SMOKING HX,0.0001,0.25,1.0,-2498.750000,0.000100,-0.000300
9,362,1046,"362, 1046",PEPTIC ULCER,0.50000,CORTICOSTEROID ADMINISTRATION PRIOR TO CURRENT...,0.4,0.0001,2.0,0.999850,0.666667,1.000050


In [104]:
# create neater version for exporting as CSV

df_final = pd.DataFrame()

df_final['Association'] = df['Dx Mx Association']
df_final['Dx Name'] = df['Dx Name']
df_final['Mx Name'] = df['Mx Name']
df_final['Prevalence'] = df['Dx Prevalence']
df_final['Import'] = df['Mx Import']
df_final['LR (+)'] = df['LR (+)']
df_final['LR (-)'] = df['LR (-)']

df_final.head()

,Association,Dx Name,Mx Name,Prevalence,Import,LR (+),LR (-)
0,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.5,2.0,1.500000,0.5000
1,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.5,2.0,0.666667,1.2000
2,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.5,2.0,0.666667,2.0000
3,"362, 198",PEPTIC ULCER,AGE 16 TO 25,0.5,3.0,0.000100,-0.0003
4,"362, 199",PEPTIC ULCER,AGE 26 TO 55,0.5,3.0,0.000100,-0.0001


In [105]:
# export df_final

df_final.to_csv('DxMxLR.csv')